In [1]:
import json
from datetime import datetime
import os, json
import pandas as pd
import numpy as np
import glob
import time
import re
from glob2 import glob

In [3]:
def read_Data(data_file, data_json, label_file):
    # open train.txt
    with open(data_file) as f:
        lines = f.readlines()
    
    jsonFiles = glob(data_json + '/*.json')
    
    # extract tweet ids from train_data.json names
    regex = re.compile('[0-9]{12,22}')
    json_lst = []
    for j in jsonFiles:   
        b=regex.findall(j)
        json_lst.append(b)
    
    data_list = {}
    num = 0
    
    # extract labels
    labels = []
    if label_file:
        with open(label_file) as f:
            label_lines = f.readlines()
        for label in label_lines:
            if label.strip('\n') == 'rumour':
                labels.append(1)
            else:
                labels.append(0)
    
    for i in range(len(lines)):
        tweets = lines[i].strip('\n').split(',')
        target_tweet = {}
        retweets = []

        if [tweets[0]] in json_lst:
    
            for j in range(len(tweets)):
                filePath = data_json+tweets[j]+'.json'
                if j == 0 :
                    json_data_file = open(filePath,'r').read()
                    json_data = json.loads(json_data_file)
                    
                    target_tweet['source tweet id']=json_data['id_str']
                    target_tweet['source tweet text']=json_data['text']
       
                if j!=0 and [tweets[j]] in json_lst:
                    json_data_file = open(filePath,'r').read()
                    json_data = json.loads(json_data_file)

                    # list the retweets in a chronongical order
                    retweets.append((time.strptime(json_data['created_at'],'%a %b %d %H:%M:%S %z %Y'), json_data['text']))
                    sort_time_retweets = sorted(retweets, key=lambda x: x[0])
                    sort_retweets = [t[1] for t in sort_time_retweets]
                    target_tweet['retweets']=sort_retweets

            data_list[num] = target_tweet
            num += 1
        elif label_file:
            labels[i] = 2
            
    labels = list(filter((2).__ne__, labels))
        
    return data_list, labels


In [5]:
train_data_file = 'train.data.txt'
train_data_json = 'train-data/'
train_label_file = 'train.label.txt'

train_data_d, train_labels=read_Data(train_data_file,train_data_json,train_label_file)

In [ ]:
print(len(train_labels))
print(len(train_data_d))

In [ ]:
# data = pd.DataFrame(train_data_d).T
# data.to_json(r'total_train_data.json')
# data_label = pd.DataFrame(train_labels).T
# data_label.to_json(r'total_train_data_labels.json')
# data

In [7]:
with open('total_train_data.json', 'w') as f:
    json.dump(train_data_d, f)
with open('total_train_data_labels.json', 'w') as f:
    json.dump(train_labels, f)

In [ ]:
dev_data_file = 'dev.data.txt'
dev_data_json = 'dev-data/'
dev_label_file = 'dev.label.txt'

dev_data_d, dev_labels=read_Data(dev_data_file,dev_data_json,dev_label_file)
print(len(dev_labels))
print(len(dev_data_d))

# dev_data = pd.DataFrame(dev_data_d).T
# dev_data.to_json(r'total_dev_data.json')
# dev_data_label = pd.DataFrame(dev_labels).T
# dev_data_label.to_json(r'total_dev_data_labels.json')
# dev_data_label
# dev_data

In [9]:
with open('total_dev_data.json', 'w') as f:
    json.dump(dev_data_d, f)
with open('total_dev_data_labels.json', 'w') as f:
    json.dump(dev_labels, f)

In [ ]:
test_data_file = 'test.data.txt'
test_data_json = 'test-data/'

test_data_d, test_labels=read_Data(test_data_file,test_data_json,"")
print(len(test_data_d))

with open('total_test_data.json', 'w') as f:
    json.dump(test_data_d, f)

In [ ]:
# combine source tweet and retweet together 

train_lst = []
for i in range(len(train_data_d)):
    tweets = train_data_d[i].get('source tweet text','')
    
    for tweet in train_data_d[i].get('retweets',''):
        tweets+=tweet
    train_lst.append(tweets)

print(len(train_lst))
print(len(train_labels))

In [ ]:
dev_lst = []
for i in range(len(dev_data_d)):
    tweets = dev_data_d[i].get('source tweet text','')
    
    for tweet in dev_data_d[i].get('retweets',''):
        tweets+=tweet
    dev_lst.append(tweets)

print(len(dev_lst))
print(len(dev_labels))